In [6]:

import os
import SimpleITK as sitk
import numpy as np
from collections import OrderedDict
from scipy.ndimage import distance_transform_edt
from scipy.interpolate import interp1d

join = os.path.join

In [51]:
marker_dir = 'marker-expert1'
save_dir = marker_dir + '_interpolated_test'
os.makedirs(save_dir, exist_ok=True)
names = sorted(os.listdir(marker_dir))
names = [name for name in names if name.endswith('.nii.gz')]

name=names[0]
nii = sitk.ReadImage(join(marker_dir, name)) 
marker_data = np.uint8(sitk.GetArrayFromImage(nii)) #(731, 512, 512)
box_data = np.zeros_like(marker_data, dtype=np.uint8)
label_ids = np.unique(marker_data)[1:]

In [53]:

def interpolate_labels(label_volume):
    depth, height, width = label_volume.shape

    interpolated_labels = np.zeros((depth, height, width), dtype=label_volume.dtype)


    # Loop through each unique label in the label volume
    for label in np.unique(label_volume):
        if label == 0:  # Skip background
            continue
        
        # Create a binary mask for the current label
        binary_mask = (label_volume == label)
        
        # Extract slices with the current label
        labeled_slices = [i for i in range(depth) if np.any(binary_mask[i])]
        
        if len(labeled_slices) < 2:  # At least two slices are needed for interpolation
            continue
        
        # Initialize array to hold distances for the slices
        distances = np.zeros((depth, height, width))
        
        # Calculate distances from object border for each labeled slice
        for i in labeled_slices:
            distances[i] = distance_transform_edt(np.logical_not(binary_mask[i]))
        
        # Create an interpolating function
        f = interp1d(labeled_slices, [distances[i] for i in labeled_slices],
                        axis=0, kind='linear', bounds_error=False, fill_value="extrapolate")
        
        # Interpolate
        for i, next_slice in zip(labeled_slices[:-1], labeled_slices[1:]):
            interpolated_slices = np.round(f(np.arange(i, next_slice + 1))).astype(int)
            
            # Update label
            interpolated_labels[i:next_slice + 1][interpolated_slices <= 0] = label

    return interpolated_labels



In [54]:
def get_bbox(mask, bbox_shift=5):
    y_indices, x_indices = np.where(mask > 0)
    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)
    # add perturbation to bounding box coordinates
    H, W = mask.shape
    x_min = max(0, x_min - bbox_shift)
    x_max = min(W, x_max + bbox_shift)
    y_min = max(0, y_min - bbox_shift)
    y_max = min(H, y_max + bbox_shift)
    bboxes = np.array([x_min, y_min, x_max, y_max])

    return bboxes

In [59]:
box_data

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [62]:
(marker_data==label_ids).astype(np.uint8)

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [56]:
for name in names:
    nii = sitk.ReadImage(join(marker_dir, name))
    marker_data = np.uint8(sitk.GetArrayFromImage(nii))
    box_data = np.zeros_like(marker_data, dtype=np.uint8)
    label_ids = np.unique(marker_data)[1:]
    print(f'label ids: {label_ids}')
    for label_id in label_ids:
        marker_data_id = (marker_data == label_id).astype(np.uint8)
        marker_zids, _, _ = np.where(marker_data_id > 0)
        marker_zids = np.sort(np.unique(marker_zids))
        print(f'z indices: {marker_zids}')
        # bbox_dict = {} # key: z_index, value: bbox
        for z in marker_zids:
            # get bbox for each slice
            z_box = get_bbox(marker_data_id[z, :, :], bbox_shift=5)
            box_data[z, z_box[1]:z_box[3], z_box[0]:z_box[2]] = label_id
    # interpolate labels
    interpolated_labels = interpolate_labels(box_data)
    # save interpolated labels
    save_name = name# .replace('.nii.gz', '_interpolated.nii.gz')
    save_path = join(save_dir, save_name)
    save_sitk = sitk.GetImageFromArray(interpolated_labels)
    # add meta information
    save_sitk.CopyInformation(nii)
    sitk.WriteImage(save_sitk, save_path)


label ids: [1]
z indices: [348 350 354 357 360 363 366 369 372 376 381 386 390 394 399 403 407 411
 418 424 434 440 445 450 454 458 463 466 468 472 476 479 483 485 487 489
 490]
label ids: [1]
z indices: [ 73  75  78  80  83  86  89  92  96  98 101 103 106 108 111 114 117 119
 121 123 124 125]
label ids: [1]
z indices: [110 114 117 123 128 134 138 142 146 151 155 157 159 160 163 164 165]
label ids: [1]
z indices: [ 48  50  52  54  57  59  64  69  73  76  81  84  88  92  97  98 102 106
 109 110 111]
label ids: [1]
z indices: [102 104 106 110 116 117 120 124 128 131 134 137 140 143 146 149 151]
label ids: [1]
z indices: [18 20 21 22 26 29 31 33 35 37 39 40 41 42]
label ids: [1 2 3]
z indices: [ 34  35  37  39  41  43  45  47  50  52  56  58  61  64  66  68  70  75
  78  80  83  86  89  92  94  96 100 103 105 106]
z indices: [41 43]
z indices: [92 94 96]
label ids: [1]
z indices: [ 86  88  92  95 100 104 109 112 115 120 124 128 130 134 136 139 142 145
 151 153 157 159 163 165 169 172 175 

In [69]:
import nibabel as nib
import numpy as np
from stl import mesh
from skimage import measure

In [71]:
file_path = 'medsam_seg_expert1/Adrenal_Ki67_Seg_010.nii.gz'

nifti_file = nib.load(file_path)
np_array = nifti_file.get_fdata()

verts, faces, normals, values = measure.marching_cubes(np_array, 0)

obj_3d = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))

for i, f in enumerate(faces):
    obj_3d.vectors[i] = verts[f]

In [72]:
obj_3d.save("Adrenal_Ki67_Seg_010.stl")